LLM 就是大语言模型 Large Language Model

生成模型有三个阶段 预训练preTrain，监督微调SFT，强化学习RLHF

工作流程：io流 当我们有对应输入，会有怎样的输出
1Tokenization 分词化
    将句子分解成更小的unit，有不同的颗粒分类，不同的语言遵循不同的方式
        词粒度Word_Level Tokenization 适用于英语类似的语言
        字符粒度Character-Level 适用于中文类似的语言
        子词粒度Subword_level 适用于有词缀词根相关的语言、
工作方式
    通过给定的token预测下一个token 按照以下recursive方式去进行预测 
        1. 基于现有tokens 预测下一个最有可能的token
        2. 将得到的token加入到现有的tokens里
        3. 重复以上操作
        4. 直到输出特殊token 如句号，end of sentence，换行符号之类的

完成数据收集和token分词后，进入到模型构建部分 主要运用到的是Transformer架构

Transformer块结构：
    Input → [LayerNorm] → [Multi-Head Attention] → [Residual Add] → [LayerNorm] → [Feedforward] → [Residual Add] → Output

    1. 输入嵌入 + 位置编码
    输入嵌入就是将Token ID转换为向量
    位置编码则是注入顺序信息

In [ ]:
    # Pseudocode
    input_embedding = embedding_layer(token_ids)  # shape: [seq_len, d_model]
    position_embedding = get_position_encoding(seq_len, d_model)
    x = input_embedding + position_embedding

    2. 多头自注意力机制
    Q = Query
    K = Key
    V = Value
    根据这三个进行计算

In [ ]:
    # Pseudocode
    Q = x @ W_q   # shape: [seq_len, d_k]
    K = x @ W_k
    V = x @ W_v
    attention_weights = softmax(Q @ K.T / sqrt(d_k))
    output = attention_weights @ V

    3. Add & LayerNorm
    将原始输入添加到注意力输出中 也就是残差连接residual connection
    规范化结果以稳定学习过程

In [ ]:
    x = LayerNorm(x + Attention(Q, K, V))

    4. 前馈网络FFN (MLP aka multiple layer perceptron 多层感知器) 两个名字都可以 取决于input和处理过程
    
    在attention之后，我们将一个简单的MLP独立的应用于每个标记的位置
    FFN(x) = ReLU(xW1 + b1)W2 + b2
    This helps model complex combinations of features per token.

    5. Final Residual + LayerNorm
    与之前相同 添加输入 然后规范化

    x = LayerNorm(x + FFN(x))

以上就是完整的一次transformer流程，通常我们将这个过程堆积12/24/96次 每一次都利用更深层次的上下文来细化表征
在最后一次迭代后 最终output会被传递到线性层 -> softmax over vocab
用于在训练期间预测下一个标记